In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd #numerical analysis library
import numpy as np #data analysis library
import matplotlib.pyplot as plt #Visualization library
import seaborn as sns #visualization library

In [ ]:
data = pd.read_csv("day.csv")

In [ ]:
# Check the head of the dataset
data.head()

Inspect the various aspects of the data dataframe

In [ ]:
data.shape
#importing dataset through pandas library

In [ ]:
data.info()
#This method prints information about a DataFrame including the index dtype and columns, non-null values and memory usage.

In [ ]:
first_column = data.pop('cnt')#putting the dependent or target variable in another variable
data.insert(0, 'cnt', first_column)#inserting the new column in new dataset
data=data.drop(['casual','registered','instant','mnth','weekday'],axis=1)#dropping all the columns based on domain knowledge, irrelavant on basis on problem statement & data dictionary

In [ ]:
data.describe()
#calculating some statistical data like percentile, mean and std of the numerical values of the data

In [ ]:
sns.pairplot(data)#pairplot command
plt.show()
#ploting multiple scatterplot for continous variable to understand there relevance relative to target variable

In [ ]:
cat_col=['season','yr','holiday','workingday','weathersit']#segrating categorical driver from dataframe for plotting purpose
cont_col=['temp','atemp','hum','windspeed']

season_dict = {1:'spring', 2:'summer', 3:'fall', 4:'winter'}#creating season a dictionary to replace 0 or 1 with corresponding categorical value
yr_dict={0:'2018',1:'2019'}#creating yr a dictionary to replace 0 or 1 with corresponding categorical value
holiday_dict={0:'Non-Holiday',1:'Holiday'}#creating holiday a dictionary to replace 0 or 1 with corresponding categorical value
workingday_dict={0:"Non-Working",1:'Working'}#creating working a dictionary to replace 0 or 1 with corresponding categorical value
weathersit_dict={1:'Clear or No Cloud',2:'Mist Cloud',3:'Light Snow or Rain',4:'Heavy Rain or Snow'}#creating a working dictionary to replace 0 or 1 with corresponding categorical value
data1=pd.DataFrame()#creating a new dataframe & adding all the above categorical values into this
data1['season']=data['season'].replace(season_dict)
data1['holiday']=data['holiday'].replace(holiday_dict)
data1['yr']=data['yr'].replace(yr_dict)
data1['workingday']=data['workingday'].replace(workingday_dict)
data1['weathersit']=data['weathersit'].replace(weathersit_dict)


In [ ]:


for i in cat_col:
    plt.figure(figsize=(12,6))#fixing the figure size of the plot
    sns.countplot(data1[i])#running command for multiple count plot for categorical variable
    plt.title("Countplot for "+i,size=17) #setting the title for each count plot
    plt.xlabel(i,fontsize=13)#fixing the label size of x axis driver
    plt.ylabel('Count',fontsize=13)##fixing the label size of y axis Count
    plt.show()

#plotting multiplecount plots for categorical variables to understand their relevance & get meaningful insights

In [ ]:
for i in cont_col:
    plt.figure(figsize=(10,5))#fixing the figure size of the plot
    plt.boxplot(data[i])#running command for multiple box plot for categorical variable
    plt.title("Boxplot for "+i,size=17) #setting the title for each box plot
    plt.xlabel(i,fontsize=15)#fixing the label size of x axis driver
    plt.show()

#ploting multiple boxplots for continous drivers to have an understanding of outliers 

In [ ]:
# Get the dummy variables for the feature season & weathersit and store it in a new variable - 'status' & 'status1'
status = pd.get_dummies(data1['season'])
status1 = pd.get_dummies(data1['weathersit'])

In [ ]:
# Let's drop the first column from status df using 'drop_first = True'

status = pd.get_dummies(data1['season'], drop_first = True)
status1 = pd.get_dummies(data1['weathersit'], drop_first = True)

In [ ]:
# Add the results to the original data dataframe

data = pd.concat([data, status], axis = 1)
data = pd.concat([data, status1], axis = 1)

In [ ]:
#the head of our dataframe.
data=data.drop(["Unnamed: 16","Unnamed: 17"],axis=1)
data.head()

In [ ]:
# Drop season & weatherit as we have created the dummies for it & holiday as we already have working day which tell 0 as weekend or holiday
data=data.drop(['weathersit','season','dteday'],axis=1)#dropping all the columns based on domain knowledge, irrelavant on basis on problem statement & data dictionary

In [ ]:
data.head()

In [ ]:
plt.figure(figsize = (14,5)) #fixing the figure size of the plot
sns.heatmap(data.corr(),annot=True,cmap='Greens',fmt='.1%',cbar=False)#running command for heatmap for correlation of continous variable
plt.show()
#correlation matrix formed through heatmap to understand the correlation of continous variables & there relevance

In [ ]:
data=data.drop(['atemp'],axis=1)#dropping atemp as it has a high correlation with temp & atemp is a feeling temperature which will be less accurate than temp

In [ ]:
from sklearn.model_selection import train_test_split #importing relevant libraries for model building

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(data, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
from sklearn.preprocessing import MinMaxScaler #importing relevant libraries for scaling the model
scaler = MinMaxScaler() #adding the scaling method in a variable


In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['temp','hum','windspeed','cnt']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (16, 10))
sns.heatmap(df_train.corr(),annot=True,cmap='Greens',fmt='.1%',cbar=False )
plt.show()

In [ ]:
# Dividing into X and Y sets for the model building
y_train = df_train.pop('cnt')
X_train = df_train

In [ ]:
from sklearn.feature_selection import RFE #importing relevant libraries to use recursive feature elimination
from sklearn.linear_model import LinearRegression #importing relevant libraries for Linear Regression modelling

In [ ]:
# Running RFE with the output number of the variable equal to 8
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, step=8)    # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))#checking the list with the top drivers & proceeding ahead with that

In [ ]:
import statsmodels.api as sm

# Add a constant or intercept & selecting temp as first feature or driver it has the highest correlation with dependent variable
X_train_lm = sm.add_constant(X_train[['temp']])

# Create a first fitted model
lr = sm.OLS(y_train, X_train_lm).fit()

In [ ]:
# Checking the parameters obtained
lr.params

In [ ]:
# visualise the data with a scatter plot and the fitted regression line
plt.scatter(X_train_lm.iloc[:, 1], y_train)
plt.plot(X_train_lm.iloc[:, 1], 0.169798 +0.639952*X_train_lm.iloc[:, 1], 'r')
plt.show()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr.summary())

In [ ]:
# Assigning additional feature variables to X
X_train_lm = X_train[['Light Snow or Rain','yr','windspeed','temp']]

In [ ]:
# Build a linear model

import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_lm)

lr = sm.OLS(y_train, X_train_lm).fit()

lr.params

In [ ]:
# Print the summary of the model
print(lr.summary())

Looking at the p-values in the above result we have it looks like the variables are really significant
And also our adjusted R2 has increase from 41% to 73%

In [ ]:
# Assigning additional feature variables to X on the basis of the REF & above correlation matrix 
X_train_lm_1 = X_train[['yr','temp','windspeed','spring','Light Snow or Rain','Mist Cloud']]


In [ ]:
# Build a linear model with new features

import statsmodels.api as sm
X_train_lm_1 = sm.add_constant(X_train_lm_1)
lr1 = sm.OLS(y_train, X_train_lm_1).fit()
lr1.params

In [ ]:
# Print the summary of the model

print(lr1.summary())

Looking at the p-values in the above result we have it looks like of the variables are really significant
And also our adjusted R2 has increase from 73% to 81%

In [ ]:
# Assigning additional feature variables to X on the basis of the REF & above correlation matrix 
X_train_lm_2 = X_train[['yr','holiday','windspeed','spring','summer','winter','Light Snow or Rain','Mist Cloud']]

In [ ]:
# Build a linear model with new features

import statsmodels.api as sm
X_train_lm_2 = sm.add_constant(X_train_lm_2)

lr2 = sm.OLS(y_train, X_train_lm_2).fit()

lr2.params

In [ ]:
# Print the summary of the model
print(lr2.summary())

Looking at the p-values in the above result we have it looks like the variables are really significant apart from working day as it has p value higher than 0.5
But our adjusted R2 has stopped improving it means we will stick to our previous model lr1 or feature in X_train_lm_1  that are ['Light Snow or Rain','yr','windspeed','temp','Mist Cloud','spring']

Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_cnt = lr1.predict(X_train_lm_1)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label
plt.show()

As we can see in the above figure our model residual is perfectly aligned at mean=0 which show its an acceptable model

In [ ]:
#Making Predictions Using the Final Model
num_vars = ['temp','hum','windspeed','cnt']
df_test[num_vars] = scaler.transform(df_test[num_vars])# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables

In [ ]:
df_test.head()

In [ ]:
# Dividing df_test set into X_test and y_test
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
# Adding constant variable to test dataframe
X_test_1 = sm.add_constant(X_test)


In [ ]:
# Creating X_test_m2 dataframe by dropping variables from X_test_m1

X_test_m2 = X_test_1.drop([ 'workingday','hum','holiday','summer','winter'], axis = 1)
X_test_m2

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred_m2)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16)  
plt.show()    

In [ ]:
# Making predictions using the lr1 which has model feature in X_train_lm_1  that are ['Light Snow or Rain','yr','windspeed','temp','Mist Cloud','spring']
from sklearn.metrics import r2_score
y_pred_m2 = lr1.predict(X_test_m2)
r2_score(y_true=y_test,y_pred=y_pred_m2)

As we can see that our R2 score on the test model is 79% which is quite similar to our training model of lr1(below) that is 81%. which shows that our model is quite stable & all the chosen driver are significant one's related to our target variable.

In [ ]:
print(lr1.summary())

### Our above choosen model lr1 has following coefficients with R2 as 81%:-
const                 0.320383- Intercept which states the value of our dependent 'cnt' when all our independent variables are kept as 0

yr                    0.236372-  year (0: 2018, 1:2019) which means 

temp                  0.362729 - temperature in Celsius as it has positive coeff which states that if the temperature is going to increase, our count of total rental bikes including both casual and registered is going to increase.

windspeed            -0.157135 - windspeed has negative coeff which states that if the windspeed is going to decrease, our count of total rental bikes including both casual and registered is going to increase.

spring               -0.153830 - spring season has negative coeff which states that as less as spring season is going to be or as ealry spring season is going to over or end, our count of total rental bikes including both casual and registered is going to increase.

Light Snow or Rain   -0.271313 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds has negative coeff which states that the less this type of weather condition is going to be,  is the more our count of total rental bikes including both casual and registered is going to increase.

Mist Cloud           -0.075476 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist has negative coeff which states that the less this type of weather condition is going to be,  is the more our count of total rental bikes including both casual and registered is going to increase.
